# Lab week 12 - Outcomes


In this lab you should read through and run the code in the lab sheet and complete the lab assessment. By the end of this lab you should be able to use R to:


* Fit a linear regression and interpret the findings.
* Use bootstrapping to approximate the distribution of regression coefficients.
* Run a permutation test to make a test decision.

Run the code in the following lines to load the necessary packages and for better graph quality.

In [ ]:
library(ggplot2)
library(repr) 
options(repr.plot.width=4, repr.plot.height=4, repr.plot.res = 120)

**Also run the next block of code please. It ensures that random sampling is reproducible. You need to re-run this block every time you restart the lab sheet!!!**

In [ ]:
## You MUST run this code (every time you open the lab sheet).
aux <- version
if (((as.numeric(aux$major) >= 3) && (as.numeric(aux$minor) >= 6)) || (as.numeric(aux$major) >= 4)) {
  RNGkind(kind = "Mersenne-Twister", normal.kind = "Inversion", sample.kind = "Rounding")
} else {
  RNGkind(kind = "Mersenne-Twister", normal.kind = "Inversion")
}

set.seed(12345)
cat("1st check: 5 = ",sample(1:6,1),"\n",sep="")
cat("2nd check: 9 = ",ceiling(runif(1,0,10)),"\n",sep="")
cat("If the statements above are right, then it is ok.\n",sep="")
cat("\n",sep="")
cat("If you get a warning message as below, this is ok. \n   'Warning message in RNGkind(kind = ",'"',"Mersenne-Twister",'"',
    ", normal.kind = ",'"',"Inversion",'"',", :\n",'   "',"non-uniform 'Rounding' sampler used",'"',"\n",sep="")

# Linear Regression


You surely remember the following datafile from a few weeks ago:

In [ ]:
galaxy <- read.csv("week12-galaxy.csv")
head(galaxy)
ggplot(galaxy, aes(x = Velocity, y = Distance)) +
 geom_point() +
 geom_smooth(method = lm, se = FALSE)

As a reminder, the two variables describe the recessional velocity (measured in km per second) of a galaxy moving away from earth and the distance of that galaxy from earth (measured in Million lightyears). 

In lab week 6 we performed a linear regression using "Velocity" as the independent and "Distance" as the dependent variable.


Recall that the regression line is the graph of the linear function $y = \beta_0 + \beta_1  x $, where $\beta_0$ and $\beta_1$ are the population parameters, so the coefficients that result from optimising the method of least squares for the entire population. Usually we don't have the data of the entire population but only a small sample at our hands. We therefore  estimate those true parameters by their sample equivalents $\hat{\beta}_0$ and $\hat{\beta}_1$, so the coefficients that result from optimising the method of least squares for the sample.

Also recall that in the equation from above the response variable $y$ depends on the explanatory variable $x$, while $\beta_0$ and $\beta_1$ are constants. The corresponding regression model is: $y_i = \hat{\beta}_0 + \hat{\beta}_1  x_i + \epsilon_i$, where $\epsilon_i$ denotes the individual vertical deviation from the regression line for the *i-th* observation. These deviations are called "residuals". 
The results of fitting a linear model to our data can be generated via the `lm()` command. Re-familiarize yourself with this command first. 

In [ ]:
# runs the linear regression and stores the residuals
lmRes <- lm(galaxy$Distance ~ galaxy$Velocity)
# summarizes the results
summary(lmRes)

Also, re-familiarize yourself with how to interpret the R output:
The first two lines show the command you ran. This is followed by a five number summary of the residuals. Next, you will find the coefficients table, this is where you will find the estimates of $\beta_0$ and $\beta_1$ and their standard errors in the first two columns. In the last column you find their corresponding p-values. Since we have studied the concept of p-values by now, we already know that they belong to a hypothesis test and yield how likely it was to observe such estimates (or more extreme ones) given the null hypothesis is true. The two presented p-values belong to the two following hypotheses pairs:

$H_0: \beta_0 = 0\,\,\, vs. H_1: \beta_0 \neq 0$ and

$H_0: \beta_1 = 0\,\,\, vs. H_1: \beta_1 \neq 0$.

Remember that the values from the table are estimates of the unknown population parameters $\beta_0$ and $\beta_1$.

The residual standard error represents the average square (vertical) distance between the observed values and the regression line measured in units of the dependent variable. You will also find the coefficient of determination $R^2$, which indicates how much of the variance of the data is explained by the model.

The relationship of "Distance" and "Velocity" could hence be described via the following equation:

$Distance = 1.603062 + 0.012152 * \, Velocity$

In the regression model we had determined a galaxy's velocity to be the predictor variable for its distance to earth. But who is to say that it doesn't work the other way around as well?

It might make sense to look at the distance to earth and use that as a predicting factor for a galaxy's velocity.

# Exercise 1: 

Exchange the two variables in the analysis from above, that is, use "Distance" as the independent and "Velocity" as the dependent variable. Fit a linear model to the data and answer the following questions.

* What is the estimated regression line? **Write it out.** 
* From the output, what are estimates for $\beta_0$, $\beta_1$ and what is$R^2$?
* From your data, does it appear that 'Velocity' depends on 'Distance'? How would you interpret the regression coefficients?
* Report the standard deviation of the residuals and interpret it. Note: This is labelled *Residual Standard Error* in the regression output.
* Is there sufficient evidence to reject the null hypothesis that the true parameter $\beta_0$ is different from 0?

Use the cell below for your analysis.

In [ ]:
# Use this space for fitting your linear model.

# Exercise 2.1: 

Now, let us repeat this method for the pregnancy datafile from last week to investigate if the duration of a pregnancy depends on the age of the mother. First, read in the 'pregnancy' datafile. Choose the dependent and the independent variable for your analysis and fit a linear model like seen above. Then answer your assignment questions.

As a reminder: The datafile contains the following variables for mother-baby pairs: The baby's birth weight in ounces, the number of gestational days, the mother's age in completed years, the mother's height in inches, pregnancy weight in pounds, and whether the mother smoked during pregnancy.

In [ ]:
pregnancy <- read.csv("week12-pregnancy.csv")
head(pregnancy)

In [ ]:
# Use this space to fit your linear model.

# Exercise 2.2 

Let's say we are interested in whether the height of the mother has an influence on the weight of the baby. To analyse this we can run a linear regression like below:

In [ ]:
lm_baby <- lm(pregnancy$Birth.Weight ~ pregnancy$Maternal.Height)
summary(lm_baby)

The estimate for the true parameter $\beta_1$ is significant at the 5% significance level. We would therefore conclude that for every additional inch of the mother's body heigt, the baby's weight increases on average by 1.478 ounces. 

Obviously our estimate of 1.478 depended on the random sample that we collected. To get an idea of the distribution of that estimate we will now bootstrap 2000 times from the original sample. Each time we will calculate the corresponding estimate for $\beta_1$. Finally, we shall visualise the empirical frequency of those estimates via a histogram.

In [ ]:
set.seed(1400)          
bootstrap_beta1 = function(replications)
{ 
  boot_beta1 = c()
  for(i in 1 : replications)
     { 
       bootstrap_sample = pregnancy[sample(nrow(pregnancy), size = nrow(pregnancy), replace = T), ]
       lm_baby = lm (bootstrap_sample$Birth.Weight ~ bootstrap_sample$Maternal.Height)
       coeff <- as.matrix(lm_baby$coefficients)
       boot_beta1[i] = coeff[2,1]
     }
  return(boot_beta1)    
}


results <- bootstrap_beta1(2000)


hist(results, xlab = "Regression Coefficient beta_1", ylab = "Density", main = "", 
     probability = T, col = "gray")


points(1.478, 0, pch = 16, cex = 1.4, col = "red")

Turn to your assignment questions now before you move on. 

Read in the 'galton' datafile next. This datafile consists of 934 datasets containing information on height and gender of family members.

In [ ]:
galton <- read.csv("week12-galton.csv")
head(galton)
dim(galton)

# Exercise 3

We would now like to investigate the linear relationship between the height of parents (measured via 'midparentHeight') and the height of their children ('childHeight'). This relationship may depend on the gender of the child, since men are on average taller than women. We will therefore have to separate our datafile (as seen below) and perform two separated analyses.

In [ ]:
galton_male <- galton[galton$gender == "male",]
head(galton_male)
nrow(galton_male)
galton_female <- galton[galton$gender == "female",]
head(galton_female)
nrow(galton_female)

Let's focus on the female children for now. Repeat the procedure presented in **Exercise 2.2**, that is:

1) Fit a linear model to the 453 observations of female children to find a point estimate for the linear relationship of the two variables of interest.

2) Bootstrap from the original sample 2000 times.

3) Visualise the empirical distribution of $\beta_1$.

The code template below will help you. 

In [ ]:
set.seed(1400)       # DO NOT remove or change this line!!! 


# no. 1) Fit the linear model for the data.

lm_galton_female <- lm(galton_female$childHeight ~ galton_female$midparentHeight)
summary(lm_galton_female)


# no. 2)  Perform the bootstrap as described above    

set.seed(1400)         # DO NOT remove or change this line even though it already appears above!!! 
bootstrap_beta1_female = function(replications)
{ 
  boot_beta1 = c()
  for(i in 1 : replications)
     { 
       boot_sample = galton_female[sample(nrow(galton_female), size = nrow(galton_female), replace = T), ]
       lm_galton_f = lm (boot_sample$childHeight ~ boot_sample$midparentHeight)
       coeff <- as.matrix(lm_galton_f$coefficients)
       boot_beta1[i] = coeff[2,1]
     }
  return(boot_beta1)    
}


results.female <- bootstrap_beta1_female(2000)


# no. 3)  Visualise the relative frequency via a histogram

hist(results.female, xlab = "Regression coefficient beta_1", ylab = "Density", main = "", 
     probability = T, col = "gray")

Now, repeat the procedure for the male children.

Note: You should copy and paste the code from above into the cell below and manipulate it accordingly.

In [ ]:
set.seed(1400)       # DO NOT remove or change this line. 

# Copy and paste the code from above into this block. Then manipulate it to answer your assignment questions.

# no. 1)

# no. 2)


set.seed(1400)         # DO NOT remove or change this line even though it already appears above!!!          

# no. 3)



## Permutation Test on Mean Difference

The last topic of this lab is "permutation test". Similarly to bootstrapping we use resampling methods in permutation tests to inform a test decision. More precisely, the exact distribution of the test statistic under the null hypothesis is obtained by calculating all possible values of the test statistic under all possible rearrangements of the observed data points. We will illustrate this via a hypothesis test on mean differences of two groups. First, load the "perth_weather" dataframe from LMS and re-familiarise yourself with it. Note that the dataframe has been reduced for the purpose of this exercise to the last 10 years only (i.e. 2000-2009) compared to previous weeks. We will aim to use the data to compare average temperatures of January and February. For this purpose, we firstly isolate and separate the information.

In [ ]:
weather =  read.csv("week12-perth_weather.csv", header = T, sep = ",")
weather.compare <- weather[weather$month==1|weather$month==2,]
head(weather.compare)

jan.temp <- weather$temp[weather$month==1]
feb.temp <- weather$temp[weather$month==2]

mean(jan.temp)
mean(feb.temp)

(mean.diff <- mean(jan.temp) - mean(feb.temp))

Based on the 10 years between 2000 and 2009 February is the warmer month. Whether the mean difference of 0.22 degrees is significant, i.e., the difference in population means is different from 0 however, is what we now strive to find out. The corresponding hypothesis test formulates as $H_0: \mu_J = \mu_F$ vs. $H_1: \mu_J \neq \mu_F$, where $\mu_J$ and $\mu_F$ are the population means of January and February temperatures respectively. 

We will now invoke a permutation test to inform a test decision. The basic idea is to compare the observed mean difference of 0.22 degrees to the distribution of mean differences for randomly assigned groups. To this end we have to repeat the following steps for all possible permutations of the group labels which yield a novel composition of the two groups: 

1) Pool all 20 observed temperatures\
2) Randomly assign 10 of them to the 'January' and the other 10 to the 'February' group\
3) Calculate the mean difference for the two groups with the randomly assigned temperatures

The procedure of generating novel permutations of group labels is as follows: Randomly select 10  numbers from the set $\{1,...,20\}$ and allocate the corresponding temperatures to the January group. The other 10 temperatures are allocated to the February group. 

The following lines of code illustrate how generating novel permutations can be achieved in R. The example considers two groups with 3 observations each. Familiarise yourself with the code below, in particular the `combinations` function, before you read on. As always: `?combinations` will help with this.

In [ ]:
source("combinations.r")
all.labels <- combinations(6, 3 , c(1:6), set=F, repeats.allowed=F)
all.labels
nrow(all.labels)

If January and February temperatures differ significantly from another the test statistic (i.e., the observed mean difference) should appear as an extreme value compared to the mean differences based on the randomly assigned groups. The following lines of code calculate the mean difference for all combinations of randomly assigned groups and produces a visualisation of the empirical distribution via a histogram.
Based on the histogram, would you consider -0.22 as an extreme observation?

In [ ]:
results <- c()
comb <- combinations(20, 10 , c(1:20), set=F, repeats.allowed=F)

for (i in 1: nrow(comb)) {
    weather.group1 <- weather.compare$temp[comb[i,]]
    weather.group2 <- weather.compare$temp[-comb[i,]]
    results[i] <- mean(weather.group1) - mean(weather.group2)
}

hist(results)
points(mean.diff, 0, pch = 16, cex = 1.4, col = "red")

The histogram represents the distribution of the test statistic under the null hypothesis. Based on the histogram, we would think that a mean difference of -0.22 does not seem to be extreme, but could very likely occur if observations were randomly assigned to either group. To quantify this observation we turn to the p-value of course.

In [ ]:
p.value <- (sum(results > abs(mean.diff)) + sum(results < -abs(mean.diff)) )/nrow(comb)
p.value

Bear in mind that we are running a two-sided hypothesis test and hence have to regard values smaller than -0.22 but also values bigger than 0.22 as extreme. Since the p-value = 0.618, we would not reject the null hypothesis at all conventional significance levels (i.e. $1\%$, $5\%$ and $10\%$ ). There is hence no sufficient evidence for either month being warmer than the other.

## Exercise 4

Replicate the above permutation test to test the hypothesis that mean temperatures in January differ from mean temperatures in December, and answer your assignment questions. You may use the code template provided in the block below.

In [ ]:
weather.compare <- ...

jan.temp <- ...
dec.temp <- ...

mean.diff <- mean(jan.temp) - mean(dec.temp)
mean.diff

results <- c()
comb <- ...

for (i in 1: nrow(comb)) {
    weather.group1 <- ...
    weather.group2 <- 
    results[i] <- ...
}

hist(results)
points(mean.diff, 0, pch = 16, cex = 1.4, col = "red")


p.value <- ...
p.value


Please finalise this week's lab sheet now by answering the last of your assignment questions. Remember to round all results to 3 decimal places. Following zero's can of course as always be omitted. For example, state 2.5 as 2.5 and not as 2.500. 

Good luck!